<a href="https://colab.research.google.com/github/lsh4205/Traffic_Simulation/blob/main/traffic_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
print("Python version:", sys.version)

Python version: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]


# Define logical 1-D domains #

The following class is a small wrapper around a 1-D Numpy array of values that define a discretized domain with a uniform step size.

In [2]:
class Domain1D:
    def __init__(self, step, min_val=-1, max_val=1):
        from numpy import linspace
        self.step = step
        n = int((max_val - min_val) / step) + 1
        self.vals = linspace(min_val, max_val, n)
        
    def __len__(self): # length in terms of # of grid points
        return len(self.vals)
    
    def __getitem__(self, I): # values by an arbitrary Numpy index-slice `I`
        return self.vals[I]

**Define a discrete time domain: $t \in [0, 120]$ with step size 0.1.**

In [3]:
T = Domain1D(0.1, min_val=0.0, max_val=120.0)
T[:]

array([0.000e+00, 1.000e-01, 2.000e-01, ..., 1.198e+02, 1.199e+02,
       1.200e+02])

**Define a discrete spatial domain: $x \in [-5, 5]$ with step size 0.01\**.

In [4]:
X = Domain1D(0.01, min_val=-5.0, max_val=5.0)
X[:]

array([-5.  , -4.99, -4.98, ...,  4.98,  4.99,  5.  ])

# Define a "solution grid" #

The following class implements a wrapper around a 2-D Numpy array that can be used to hold $u_{j, i} = u(x_i, t_j)$.

> The indices of $u_{j,i}$ are transposed relative to the "whiteboard" notes.

When instantiated, a `Solution1D` object includes "padding" in space to hold "ghost zones," which are copies of the endpoints that can help simplify how boundary conditions are handled.

In particular, suppose the desired solution domain at discrete time-index $j$ _logically_ consists of the $n$ points $u_{j,0}, u_{j,1}, u_{j,2}, \ldots, u_{j,n-1}$. The `Solution1D` object creates a _physical_ array of size $n+2$, which has extra cells to the left and right of the logical values, as shown below.

$$\begin{array}{r|c|l}
  a_j & u_{j,0} \: u_{j,1} \: \cdots \: u_{j, n-1} & b_j
\end{array}$$

The entries $a_j$ and $b_j$ are the **ghost values**. The function `Solution1D.update_ghosts(j)` will set $a_j \leftarrow u_{j,n-1}$ and $b \leftarrow u_{j,0}$, which correspond to **periodic** or **wraparound** boundary conditions.

In [5]:
class Solution1D: # with periodic "ghost boundaries"
    def __init__(self, T, X, u0=None):
        from numpy import zeros
        assert isinstance(T, Domain1D) and isinstance(X, Domain1D)
        self.T = T
        self.X = X
        self.vals = zeros((len(T), len(X) + 2)) # ghost cells
        
        if u0 is not None and len(T) > 0:
            self.vals[0, 1:-1] = u0(X) if callable(u0) else u0
            self.update_ghosts(0)
            
    def __getitem__(self, s): # values by arbitrary Numpy-style slice
        assert isinstance(s, tuple) and len(s) == 2
        J, I = s[0], s[1]
        return self.vals[J, I]
    
    def __setitem__(self, s, x):
        assert isinstance(s, tuple) and len(s) == 2
        J, I = s[0], s[1]
        self.vals[J, I] = x
        
    def __len__(self):
        return len(self.T)
    
    def update_ghosts(self, j): # periodic boundaries
        self.vals[j+1, 0] = self.vals[j, -2]
        self.vals[j+1, -1] = self.vals[j, 1]